## Day 18: Duet

http://adventofcode.com/2017/day/18

### Part 1

This isn't going to be pretty. I can't think of a way to do this with proper functions without passing state around everywhere, so let's just have one massive function.

In [1]:
from collections import defaultdict

def run(commands):
    registers = defaultdict(int)

    # Returns a number if it's a number or registers[x] otherwise
    def value(x):
        return registers[x] if x.isalpha() else int(x)

    last_sound = None
    command_number = 0
    
    while(0 <= command_number < len(commands)):
        #print(registers, commands[command_number])
        fields = commands[command_number].split()
        command_name = fields[0]
        parameters = fields[1:]
        
        x = parameters[0]
        if command_name == 'snd':
            last_sound = registers[x]
        elif command_name == 'rcv':
            if last_sound != 0:
                return last_sound
        else:
            y = parameters[1]
            if command_name == 'set':
                registers[x] = value(y)
            elif command_name == 'add':
                registers[x] += value(y)
            elif command_name == 'mul':
                registers[x] *= value(y)
            elif command_name == 'mod':
                registers[x] = registers[x] % value(y)  
            elif command_name == 'jgz':
                if value(x) > 0:
                    command_number += value(y) - 1
                    
        command_number += 1

In [2]:
test_code = '''set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2'''.splitlines()

run(test_code)

4

That works, but the number's so small it could be luck.

In [3]:
with open('input', 'r') as f:
    input_code = f.readlines()
    
run(input_code)

3188

Yes, it was! The command line number wasn't being updated correctly.

### Part 2

Good grief, I'll need a think about this one. I might read up on asyncio. (Much later...) I think you might be able to do something with coroutines but there's only so much time I'm willing to spend on this, let's just hack something out. Look away now.

This goes step by step for each line of the program, waiting if there are no sounds to receive.

In [4]:
from collections import deque

def run_part_2(commands):
    registers = [defaultdict(int, {'p': program_id}) for program_id in (0, 1)]
    command_numbers = [0, 0]
    waiting_sounds = [deque() for _ in (0, 1)]
    waiting = [False, False]
    num_sounds = [0, 0]

    # Returns a number if it's a number or registers[p][x] otherwise
    def value(x, p):
        return registers[p][x] if x.isalpha() else int(x)
    
    def stopped():
        return all(waiting) \
            or waiting[0] and not 0 <= command_numbers[1] < len(commands) \
            or waiting[1] and not 0 <= command_numbers[0] < len(commands)
    
    # Run while either program is still going
    while not stopped():
        for p in (0, 1):
            if 0 <= command_numbers[p] < len(commands): 
                fields = commands[command_numbers[p]].split()
                command_name = fields[0]
                parameters = fields[1:]
        
                x = parameters[0]
                if command_name == 'snd':
                    waiting_sounds[p^1].append(value(x, p))
                    num_sounds[p] += 1
                elif command_name == 'rcv':
                    if waiting_sounds[p]:
                        registers[p][x] = waiting_sounds[p].popleft()
                        # print(str(p) + " received " + str(registers[p][x]))
                        waiting[p] = False
                    else: 
                        waiting[p] = True
                        command_numbers[p] -= 1
                else:
                    y = parameters[1]
                    if command_name == 'set':
                        registers[p][x] = value(y, p)
                    elif command_name == 'add':
                        registers[p][x] += value(y, p)
                    elif command_name == 'mul':
                        registers[p][x] *= value(y, p)
                    elif command_name == 'mod':
                        registers[p][x] = registers[p][x] % value(y, p)  
                    elif command_name == 'jgz':
                        if value(x, p) > 0:
                            command_numbers[p] += value(y, p) - 1
                    
                command_numbers[p] += 1
                
    return num_sounds[1]
        

In [5]:
test_code = '''snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d'''.splitlines()

run_part_2(test_code)

3

In [6]:
run_part_2(input_code)

7112